In [1]:
'''
Strategy 3: From Literature: Adaptive Trading strategy
'''

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from backtesting import Backtest, Strategy
from datetime import datetime
import talib
import joblib
import matplotlib.pyplot as plt

# Load data
data = pd.read_csv('./EURUSD_D1.csv')
data['Time'] = pd.to_datetime(data['Time'], format='%Y-%m-%d %H:%M:%S')
data.set_index('Time', inplace=True)


#### Feature Engineering ####
def multivariateFeatureEngineering(data):

    data['50_sma'] = data['Close'].rolling(window=50).mean() 
    data['200_sma'] = data['Close'].rolling(window=200).mean() 
    data['50_ema'] = data['Close'].ewm(span=50, adjust=False).mean()
    data['100_ema'] = data['Close'].ewm(span=100, adjust=False).mean()
    data['12_ema'] = data['Close'].ewm(span=12, adjust=False).mean()
    data['26_ema'] = data['Close'].ewm(span=26, adjust=False).mean()
    data['MACD_line'] = data['12_ema']-data['26_ema'] # calculate the MACD line
    data['Signal_line'] = data['MACD_line'].ewm(span=9, adjust=False).mean() # 9-preiod ema signal calculated from the Macdline
    data['ADX'] = talib.ADX(data['High'], data['Low'], data['Close'], timeperiod=14)
    data['RSI'] = talib.RSI(data['Close'], timeperiod=14)
    data['stoch_k'], data['stoch_d'] = talib.STOCH(data['High'], data['Low'], data['Close'], 
                                                fastk_period=14, slowk_period=3, slowd_period=3)
    data['ATR'] = talib.ATR(data['High'], data['Low'], data['Close'], timeperiod=14)
  
    data = data.dropna() # drop rows that have NA
    data = data.drop(columns=['12_ema', '26_ema'])

    return data

###### Generate lag feaures #######
def multivariateFeatureLagMultiStep(data, n_past, future_steps, target_column):
    features = []
    response = []

    max_future_step = max(future_steps)
    num_features = data.shape[1]
    group_feature_lags =  1 # change grouping of lagged features

    # Adjust the loop to prevent index out of bounds
    for i in range(n_past, len(data) - max_future_step + 1):

        if group_feature_lags==1:
                
            lagged_features = []

            for feature_idx in range(num_features):
                feature_lags = data.iloc[i - n_past:i, feature_idx].values 
                lagged_features.extend(feature_lags) 

        elif group_feature_lags==0:
            features.append(data.iloc[i - n_past:i, :].values)  # Take all columns as features

        # Use .iloc for integer-based indexing and .values to get a NumPy array

        if group_feature_lags==1:
            features.append(lagged_features)

        # Extract the target values at specified future steps using .iloc
        response.append([data.iloc[i + step - 1, target_column] for step in future_steps])

    # Convert lists to NumPy arrays after the loop
    features = np.array(features)  # Shape: (num_samples, n_past, num_features)
    response = np.array(response)  # Shape: (num_samples, len(future_steps))

    # Flatten the features to 2D array: (num_samples, n_past * num_features)
    features_flat = features.reshape(features.shape[0], -1)

    return features_flat, response


############################# Load saved Best model information ##################################################

best_model_info= [1, 1, ['Open', 'Low', 'sma_50', 'sma_200', 'ema_50', 'MACD_line', 'Signal_line', 'RSI', 'ATR', 'ema_100', 'High', 'Close']]

best_model_data = joblib.load('./model _weights/best_model_weights_and_scaler.pkl')
scaler = best_model_data['scaler']
weights = best_model_data['weights']
bias = best_model_data['bias']

lookback_window = best_model_info[0]
features = best_model_info[2]

data = multivariateFeatureEngineering(data) # generate additional features
#rename columns 
data = data.rename(columns={
    '50_sma': 'sma_50',
    '200_sma': 'sma_200',
    '50_ema': 'ema_50',
    '100_ema': 'ema_100'

})

################################# # Backtesting Strategy using Linear Regression #########################################

class LinearRegressionStrategy(Strategy):
    profit_threshold = 130
    short_sl_multiplier =1.01 
    short_tp_multiplier = 0.98 
    long_tp_multiplier = 1.02
    long_sl_multiplier = 0.99

    def init(self):
        self.index = 1         # Track positions and predictions
        self.test_data_subset = self.data.df[features]        # Prepare test data subset with the best selected features
        self.Trend = 0
    def forecast_prices(self, data_window):

        """Forecast prices for 1-day, 3-day, and 5-day horizons."""
        num_features = data_window.shape[1]         # Get the number of features

        lagged_features = []         # Flatten the data with grouping by feature type

        for feature_idx in range(num_features):
            feature_lags = data_window.iloc[:, feature_idx].values             # Extract all lagged values for the current feature

            lagged_features.extend(feature_lags)

        X = np.array(lagged_features).reshape(1, -1)         # Convert to a NumPy array and reshape to fit the model's input (flattening)


        X_scaled = scaler.transform(X)         # Scale the flattened features using the trained scaler

        # Forecast using the model's weights and bias
        predictions = np.dot(X_scaled, weights) + bias  # Shape: (1, 3)

        return predictions[0]  #    
    
    ######### Strategy Exectution #############
    def validate_order(self, trade_type, order_price, sl, tp):
        """
        Validates the order to ensure the correct relationship between TP, order price, and SL.
        - For long: TP > order price > SL
        - For short: TP < order price < SL
        """

        adjust_order =  0
        if trade_type == 'long':
            adjust_order = 1
            if not (tp > order_price > sl):
                print(f"CHECK: Long orders require: TP ({tp}) > Order ({order_price}) > SL ({sl})")
                return adjust_order 

        elif trade_type == 'short':
            adjust_order = 2
            if not (tp < order_price < sl):
                print(f"CHECK: Short orders require: TP ({tp}) < Order ({order_price}) < SL ({sl})")

                return adjust_order

    
    def adjust_order_limits(self, trade_type, order_price, sl, tp):
        if trade_type == 'long':
            if tp > order_price:
                tp  =tp 
            elif tp < order_price:
                tp = order_price + 0.016 
            
            if sl < order_price:
                sl =  sl
            elif sl > order_price:
                sl =  order_price - 0.008
          
        elif trade_type == 'short':
            if sl > order_price:
                sl  = sl 
            elif sl < order_price:
                sl = order_price + 0.008

            if tp < order_price:
                tp = tp 

            elif tp > order_price:
                tp  =  order_price - 0.016
           
        return sl, tp
    
    def enter_trade(self, forecast_1d, forecast_3d, forecast_5d, current_close):
        """
        Enter a trade only if there is no active position.
        """
        if not self.position:  # Check if there is no active position
            if forecast_5d > forecast_3d > forecast_1d > current_close or forecast_3d>forecast_1d>current_close:
                # print(f'Entering long trade at {current_close}')
                self.buy(size=0.1, sl=self.long_sl_multiplier * current_close, tp=self.long_tp_multiplier * current_close)  # Open a long position

            else:
                # print(f'Entering short trade at {current_close}')
                self.sell(size=0.1, sl=self.short_sl_multiplier* current_close, tp=self.short_tp_multiplier* current_close)  # Open a short position
        else:
            # print(f'Trade still active. Waiting for it to close.')
            print('')


    def set_sl_tp(self, current_close, forecast_3d):
        stop_loss_distance = 0.008  # 10 pips distance

        if self.position.is_long:
            sl = current_close - stop_loss_distance  # SL below entry for long position
            tp = forecast_3d + 0.01 # Use 1-day forecast as TP

        elif self.position.is_short:
            sl = current_close + stop_loss_distance  # SL above entry for short position
            tp = forecast_3d - 0.01 # Use 1-day forecast as TP

        else:
            # Fallback values if there is no active position
            sl = current_close - stop_loss_distance  # Assume a hypothetical long setup
            tp = current_close + stop_loss_distance  # Use a small profit target

        return sl, tp

    
    # def scale_position(self, forecast_3d, current_close):
    #     if self.position.is_long and forecast_3d > current_close:
    #         self.buy(size=0.05, sl=self.long_sl_multiplier * current_close, tp=self.long_tp_multiplier * current_close)    # Add to the long position

    #     elif self.position.is_short and forecast_3d < current_close:
    #         self.sell(size=0.1, sl=self.short_sl_multiplier* current_close, tp=self.short_tp_multiplier* current_close)    # Add to the short position

            
    def update_trailing_stop(self, current_close):                                                                                     #?? How does this apply to multiple positions?
        stop_loss_distance = 0.008  # 30 pips distance                                                                                     Longer trailing SL

        if self.position.is_long:
            new_sl = current_close - stop_loss_distance
            self.position.sl = max(self.position.sl, new_sl)  # Adjust SL upward                                                             ??? correct sl access

        elif self.position.is_short:
            new_sl = current_close + stop_loss_distance
            self.position.sl = min(self.position.sl, new_sl)  # Adjust SL downward                                                           ??? correct sl access


    def exit_trade(self, forecast_1d, current_close):
        if self.position.is_long and current_close >= forecast_1d:                                                             #             ?? Should I perform this check based 5d forecast and shoudl trade not be closed entirely
            self.position.close(portion=0.5)  # Close 50% of the position
            # self.position.close()  # Close 50% of the position

        elif self.position.is_short and current_close <= forecast_1d:
            self.position.close(portion=0.5)  # Close 50% of the position
            # self.position.close()  # Close 50% of the position

    def check_and_close_trade(self):
        """
        Check if the current trade's profit exceeds the threshold. If yes, close the trade.
        """
        if self.position and self.position.pl >= self.profit_threshold:
            # print(f"Closing trade early with profit: {self.position.pl}")
            self.position.close()  # Close the trade




    def next(self):
        if self.index >= lookback_window:
            # Get the latest data window
            data_window = self.test_data_subset.iloc[self.index - lookback_window + 1 : self.index + 1]

            # Forecast prices for 1-day, 3-day, and 5-day horizons
            forecast_1d, forecast_3d, forecast_5d = self.forecast_prices(data_window)

            # Get the current close price (entry price)
            current_close = self.data.Close[self.index]

            self.check_and_close_trade()

            # Enter trades based on forecasts
            self.enter_trade(forecast_1d, forecast_3d, forecast_5d, current_close)

            # Set stop loss and take profit levels
            # sl, tp = self.set_sl_tp(current_close, forecast_3d)

            # Apply the stop loss and take profit to the current position
            # if self.position.is_long or self.position.is_short:
            #     self.position.sl = sl
            #     self.position.tp = tp

            # Scale the position if conditions are favorable
            # self.scale_position(forecast_3d, current_close)

            # # Update the trailing stop to lock in profits
            # self.update_trailing_stop(current_close)

            # # Check if we need to partially exit the trade
            # self.exit_trade(forecast_1d, current_close)
            print(self.index)

        self.index += 1


test_data = data.iloc[-481:]  # This assumes X_test is at the end of the dataset
bt = Backtest(test_data, LinearRegressionStrategy, cash =10000, commission=.002, margin=.05, trade_on_close=True)
stats =  bt.run()
# stats = bt.optimize(
#     profit_threshold=range(30,150, 10),  # Test different threshold values
#     short_sl_multiplier=[1.01, 1.02, 1.03,1.04],  # Test different SL multipliers
#     short_tp_multiplier=[0.97, 0.98, 0.99, 0.995],  # Test different TP multipliers
#     long_sl_multiplier=[0.97, 0.98, 0.99, 0.995],  # Test different SL multipliers
#     long_tp_multiplier=[1.01, 1.02, 1.03,1.035],  # Test different TP multipliers
#     maximize='Equity Final [$]',  # Choose performance metric to maximize
# )

bt.plot() # trade execution

stats # trade statistics

best_params = stats._strategy._params

print(best_params)



/home/dgibrilly/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1

2

3

4
5

6

7

8

9

10

11
12

13

14

15
16

17

18

19

20

21

22
23

24

25

26

27

28

29

30

31

32

33

34

35
36

37

38

39

40

41
42

43

44

45

46

47

48

49

50

51

52
53

54

55

56

57

58

59

60

61

62

63
64

65

66
67

68

69

70

71

72

73

74

75

76

77

78

79

80

81

82

83

84

85

86

87

88

89
90

91

92

93

94

95

96

97
98

99

100

101
102

103

104

105

106

107

108

109

110

111

112

113

114

115

116

117

118

119
120

121

122

123

124

125

126

127

128

129

130

131

132

133
134

135

136

137

138

139

140

141

142

143

144
145

146

147

148

149

150

151

152

153

154

155
156

157

158

159

160
161

162

163

164

165
166

167

168

169
170

171

172

173

174

175

176
177

178

179
180

181

182

183
184

185

186

187
188

189

190

191

192

193

194

195

196
197

198

199

200

201
202

203

204

205

206

207

208

209

210

211

212
213

214

215

216

217

218

219

220

221
222

223
224

225

226

227
22